In [ ]:
import pandas as pd
import requests
import zipfile
import io
import plotly.graph_objects as go
import streamlit as st  # Se añade para mostrar la gráfica
import sys

# --- INICIALIZACIÓN DE PARÁMETROS ---
COLOR_HOMBRES = "#889064"  # Verde corporativo (Default)
COLOR_MUJERES = "#0b0d0e"  # Negro (Default)
FONT_FAMILY = "Aptos Light"  # (Default)
municipio_query = None

# Parámetros inyectados desde Streamlit
if 'active_palette' in locals() and len(active_palette) >= 2:
    COLOR_HOMBRES = active_palette[0]
    COLOR_MUJERES = active_palette[1]

if 'active_font' in locals():
    FONT_FAMILY = active_font

if 'MUNICIPIO_SELECCIONADO' in locals():
    municipio_query = MUNICIPIO_SELECCIONADO.strip().lower()

if not municipio_query:
    st.error("❌ Error: No se ha seleccionado un municipio en la aplicación Streamlit.")
    sys.exit(1)

# ====== Descarga y lectura del CSV INEGI ======
url = "https://www.inegi.org.mx/contenidos/programas/ccpv/2020/datosabiertos/iter/iter_00_cpv2020_csv.zip"

st.info("Descargando archivo desde INEGI...")

try:
    response = requests.get(url, timeout=30)
    if response.status_code == 200:
        st.info("Descarga completa. Extrayendo archivo...")
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            target_file = None
            for file_name in z.namelist():
                if "conjunto_de_datos_iter_00CSV20.csv" in file_name:
                    target_file = file_name
                    break

            if target_file:
                st.info(f"Archivo encontrado: {target_file}")
                with z.open(target_file) as f:
                    df = pd.read_csv(f, encoding="latin-1", skipinitialspace=True)
            else:
                st.error("No se encontró el archivo CSV esperado dentro del ZIP.")
                sys.exit(1)
    else:
        st.error(f"Error al descargar el archivo. Código: {response.status_code}")
        sys.exit(1)

except requests.exceptions.RequestException as e:
    st.error(f"Error de conexión al descargar el archivo: {e}")
    sys.exit(1)

except Exception as e:
    st.error(f"Ocurrió un error inesperado durante la descarga/extracción: {e}")
    sys.exit(1)

# ====== Limpieza ======
df.columns = [col.replace("ï»¿", "").strip() for col in df.columns]

for col in ["NOM_ENT", "NOM_MUN"]:
    df[col] = df[col].apply(
        lambda x: str(x).encode("latin-1").decode("utf-8") if isinstance(x, str) else x
    )

df["LOC"] = pd.to_numeric(df["LOC"], errors="coerce").fillna(-1).astype(int)
df_municipios = df[df["LOC"] == 0].copy()

for col in ["POBTOT", "POBFEM", "POBMAS"]:
    df_municipios[col] = pd.to_numeric(df_municipios[col], errors="coerce")

# RANGOS DE EDAD
rangos = [
    ("0 a 4 años", "P_0A4_F", "P_0A4_M"),
    ("5 a 9 años", "P_5A9_F", "P_5A9_M"),
    ("10 a 14 años", "P_10A14_F", "P_10A14_M"),
    ("15 a 19 años", "P_15A19_F", "P_15A19_M"),
    ("20 a 24 años", "P_20A24_F", "P_20A24_M"),
    ("25 a 29 años", "P_25A29_F", "P_25A29_M"),
    ("30 a 34 años", "P_30A34_F", "P_30A34_M"),
    ("35 a 39 años", "P_35A39_F", "P_35A39_M"),
    ("40 a 44 años", "P_40A44_F", "P_40A44_M"),
    ("45 a 49 años", "P_45A49_F", "P_45A49_M"),
    ("50 a 54 años", "P_50A54_F", "P_50A54_M"),
    ("55 a 59 años", "P_55A59_F", "P_55A59_M"),
    ("60 a 64 años", "P_60A64_F", "P_60A64_M"),
    ("65 a 69 años", "P_65A69_F", "P_65A69_M"),
    ("70 a 74 años", "P_70A74_F", "P_70A74_M"),
    ("75 a 79 años", "P_75A79_F", "P_75A79_M"),
    ("80 a 84 años", "P_80A84_F", "P_80A84_M"),
    ("85 y más", "P_85YMAS_F", "P_85YMAS_M"),
]

# FILTRO
resultados = df_municipios[
    df_municipios["NOM_MUN"].str.lower().str.contains(municipio_query, na=False)
]

# ====== FUNCIÓN PIRÁMIDE ======
def construir_piramide(fila, color_hombres, color_mujeres, font_family):

    edades = []
    mujeres = []
    hombres = []

    for etiqueta, col_f, col_m in rangos:
        val_f = pd.to_numeric(fila.get(col_f), errors="coerce")
        val_m = pd.to_numeric(fila.get(col_m), errors="coerce")

        val_f = 0 if pd.isna(val_f) else int(val_f)
        val_m = 0 if pd.isna(val_m) else int(val_m)

        edades.append(etiqueta)
        mujeres.append(val_f)
        hombres.append(-val_m)

    fig = go.Figure()

    fig.add_trace(go.Bar(
        y=edades,
        x=hombres,
        name="Hombres",
        orientation="h",
        marker_color=color_hombres,
        hovertemplate="<b>Hombres</b>: %{customdata:,}<extra></extra>",
        customdata=[abs(x) for x in hombres]
    ))

    fig.add_trace(go.Bar(
        y=edades,
        x=mujeres,
        name="Mujeres",
        orientation="h",
        marker_color=color_mujeres,
        hovertemplate="<b>Mujeres</b>: %{x:,}<extra></extra>"
    ))

    max_val = max(max(mujeres), max([abs(x) for x in hombres]))
    step = max(round(max_val / 6), 1)

    left_vals = [-i for i in range(0, max_val + step, step)]
    right_vals = [i for i in range(0, max_val + step, step)]

    tickvals = left_vals + right_vals[1:]
    ticktext = [f"{abs(i):,}" for i in left_vals] + [f"{i:,}" for i in right_vals[1:]]

    fig.update_layout(
        title=dict(
            text=f"Pirámide poblacional - {fila['NOM_MUN']}, {fila['NOM_ENT']}",
            x=0.5,
            xanchor='center' # <--- CAMBIO 1: Título centrado
        ),
        barmode="overlay",
        bargap=0.1,
        xaxis=dict(
            title="Población",
            tickvals=tickvals,
            ticktext=ticktext,
            zeroline=True,
            zerolinewidth=2,
            zerolinecolor="#555",
        ),
        yaxis=dict(title="Edad"),
        font=dict(family=font_family, size=14),
        template="plotly_white",
        legend=dict(orientation="h", y=-0.2, x=0.5, xanchor="center"),
        # <--- CAMBIO 2: Margen inferior aumentado
        margin=dict(b=150, l=50, r=50)
    )

    # <--- CAMBIO 3: Leyenda de Fuente
    fig.add_annotation(
        text="Fuente: INEGI (Censo de Población y Vivienda 2020)",
        xref="paper", yref="paper",
        x=0, y=-0.3,
        showarrow=False,
        xanchor='left', yanchor='top',
        font=dict(size=12, color="gray", family=font_family)
    )

    return fig

# ====== MOSTRAR RESULTADOS ======
if not resultados.empty:
    st.subheader(f"Resultados para: **{municipio_query.title()}**")

    for _, fila in resultados.iterrows():
        pobtot = fila["POBTOT"]
        pobfem = fila["POBFEM"]
        pobmas = fila["POBMAS"]

        st.markdown(f"#### Estado: {fila['NOM_ENT']} | Municipio: {fila['NOM_MUN']}")

        colA, colB, colC = st.columns(3)
        colA.metric("Población Total", f"{pobtot:,}")

        if pd.notna(pobfem) and pd.notna(pobmas) and pobtot > 0:
            colB.metric("Mujeres", f"{pobfem:,}")
            colC.metric("Hombres", f"{pobmas:,}")
        else:
            st.warning("Datos incompletos de sexo.")

        st.markdown("---")

        fig = construir_piramide(fila, COLOR_HOMBRES, COLOR_MUJERES, FONT_FAMILY)
        st.plotly_chart(fig, use_container_width=True)

else:
    st.warning(f"⚠️ No se encontraron coincidencias para '{municipio_query.title()}'.")